In [1]:
import os

In [2]:
%pwd

'd:\\Visa-Approval-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Visa-Approval-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.VisaApprovalPredict.constants import *
from src.VisaApprovalPredict.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from src.VisaApprovalPredict import logger
from src.VisaApprovalPredict.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-03 23:52:55,753: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-03 23:52:55,760: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-03 23:52:55,771: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-03 23:52:55,773: INFO: common: Directory already exists at: artifacts]
[2025-01-03 23:52:55,773: INFO: common: Created directory at: artifacts/data_ingestion]


[2025-01-03 23:52:58,863: INFO: 2280763589: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 341902
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d5d04f2447972a8a2910e4551c5b46f30b1f88d7746581096a5025fb0573b5c2"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9EA8:3485BF:4BFF86E:5286A31:67792118
Accept-Ranges: bytes
Date: Sat, 04 Jan 2025 11:52:58 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990021-FJR
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1735991578.184030,VS0,VE402
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 0e3176c311241bdd60ed5d4a10764ec783f6852f
Expires: Sat, 04 Jan 2025 11:57:58 GMT
Source-Age: 0

]
